# Trabajo Final LP2

## Integrantes:

- Poma Gómez Diego Alonso
- Giancarlo Kuway
- Manuel Mercedez

In [15]:
# Librerias a instalar 

#pip install pretty-html-table
#pip install selenium

# Descargar
public='C:/Users/gian_/OneDrive/Escritorio/Trabajo_Final_LP2/chromedriver.exe' 

In [56]:
# Ingresa producto y correo electrónico
producto= input("Ingrese el producto a comparar: ")
correo= input("Ingrese su dirección de correo electrónico (ej: example@gmail) ")

Ingrese el producto a comparar: avena
Ingrese su dirección de correo electrónico (ej: example@gmail) gijakuche@ggmail.com


# Metro

## Integrante:

In [17]:
# PRUEBA FINAL (todos los productos de Metro)

# Se importan librerias
from selenium.webdriver.common.keys import Keys                  # Para cambiar el nombre de un producto inválido ingresado
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
                                                                          # deprecated .... iperkenel.
driver = webdriver.Chrome(public,options = options)

# Se asigna la pagina web a scrapear
driver.get("https://www.metro.pe/") 

# Da click para aceptar términos de un aviso emergente
WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.swal2-confirm swal2-styled'.replace(' ','.')))).click()

# Selecciona y escribe el producto en el buscador Metro 
WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(producto)


# Se verifica si el nombre del producto ingresado es válido

while True:
    try:
        # Click en 'Ver todos los resultados'
        WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'a.see-all'.replace(' ','.')))).click()
        break
    except:
        print("No se encontraron resultados, nombre de producto inválido")
        producto=input('Vuelva a ingresar el nombre de producto: ')
        # Se sombrea el texto del producto inválido
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(Keys.CONTROL + 'a')
        sleep(1)
        # Se escribe el nuevo nombre de producto ingresado
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#search-autocomplete-input'))).send_keys(producto)
        
sleep(1)
# Hace scroll directo hacia el final de página
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")     

# Espera a que carguen todos los productos de la página

# n_product es la cantidad de productos encontrados 
n_product=driver.find_element('xpath','//span[@class="amount"]').text
n_product=n_product.split()
# Si la cantidad de productos es mayor a 18 , esperar a que cargue la página
if int(n_product[0])>18:
    WebDriverWait(driver, 120).until(
        EC.presence_of_all_elements_located((
            By.XPATH,'//div[@class="shelf-content"]/div[@style="display: none;"]')))  

# Se asigna elementos a cada producto
elementos= driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]')

datos1=[]
# Se extraen el nombre(junto con la cantidad) y el precio
for elem in elementos:
    saco=[]
    try:
        precio= elem.find_element("xpath",'.//span[@class="product-prices__value product-prices__value--best-price"]').text
        nombre= elem.find_element("xpath",'.//a[@class="product-item__name"]').text
        print(nombre)
        print(precio)
        saco.append(nombre)
        saco.append(precio)
        datos1.append(saco)
    except:
        precio = 'NO DISPONIBLE'
        producto = 'NO DISPONIBLE'

<ipython-input-17-4a58bb3c9e53>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(public,options = options)


Avena Natural Santa Catalina 900g
S/. 11.99
Avena Con Manzana Instantánea Santa Catalina Bolsa 150 g
S/. 4.79
Avena Tradicional Cuisine&Co Bolsa 300 g
S/. 4.15
Avena con Algarrobina Santa Catalina 280g
S/. 5.39
Avena Cereal Andino 270 g
S/. 5.90
Avena Con Quinua Quaker Bolsa 360 g
S/. 6.80
Avena Premium Bolsa 300 g
S/. 5.90
Avena Instantánea con Coco y Plátano Sin Gluten Dyfferent Pote 65 g
S/. 5.49
Avena Alpina Deslactosada Vaso 250 g
S/. 3.50
Avena Instantánea Natural Caja 350 g
S/. 8.99
Avena Precocida 3 Ositos 900g
S/. 11.90
Avena Grano de Oro 1kg
S/. 8.90
Avena Grano de Oro 500g
S/. 4.70
Avena Tradicional Quaker 200 g
S/. 4.50
Avena Clasica 3 Ositos Bolsa 10 kg
S/. 30.90
Avena con Maca Quaker Bolsa 360 g
S/. 6.39
Avena Precocida Con Chía y Algarrobo Bolsa 360 g
S/. 6.80
Avena Precocida 3 Ositos 500g
S/. 8.90
Avena Instantánea con Manzana y Canela Sin Gluten Dyfferent Pote 65 g
S/5.49
Avena Con Chocolate Instantáneo Santa Catalina Bolsa 150 g
S/4.79
Avena Multisemillas Linaza, Cañi

In [18]:
# LIMPIEZA Y ESTRUCTURACIÓN DE DATOS DESCARGADOS (METRO)

# Se crea la columna Cantidad
# Se importan librerias
import re
import pandas as pd

total1=[]

for i in datos1:
    saco1=[]   # nombre-precio-cantidad
    saco2=[]   # cantidad
    saco3=[]   # Agrupa los datos de cada producto
    saco1.append(i[0])
    for e in i:
        if e==i[0]:
            x= re.findall('\sx\s',e)    # Posee letra _x_ ?
            y= re.findall('[Pp]ack',e) # Posee palabra P-pack en el texto ?
            z= re.findall('^Mix',e)     # Posee letra Mix al comienzo ?
            w=re.findall('\sx.+',e)     # Coincide desde x_ hasta el final de la oración
            o= re.findall('\s[/]\s',e)  # Posee símbolo '/' 
            u= re.findall('\s[+]\s',e)  # Posee símbolo '+'
            q=re.findall('\d.+',e)      # Coincide con cualquier numero_ hasta el final de la oración
            oq=re.findall('\d+',e)      # Coincide con todos los números enteros
            if len(y)>0:
                saco2.append('Pack') #se almacena pack como cantidad
            elif len(z)>0:
                saco2.append('Mix') # se almacena Mix como cantidad

            elif len(x)>0:
                if len(w)>0:
                    saco2.append(w[0])

            elif len(x)==0:
                if len(oq)==0:
                    if len(u)>0:
                        saco2.append('Duo')
                    else:
                        saco2.append('1 unidad')
                elif len(o)>0:
                    saco2.append(q[0])
                elif len(u)==0:
                    v=re.findall('\d+\s+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)',e)
                    v1=re.findall('\d+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)',e)
                    # v y v1 sirven para encontrar condiciones en la oracion 'e'
                    if len(oq)>=2:
                        t2=re.findall('%',e)
                        lp=e.split()    # Separa la oración en palabras
                        a=0     # contador
                        for elem in lp:
                            t=re.findall('^\d.+([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)$',elem)
                            t1=re.findall('^([Mm]l|[Kk]g|g|[Uu]nid|[Ll]t|[Ll]|[Ss]obres|gr|[Uu]n\.|[Uu]nidades|Filtrantes|[Ll]itro|[Ll]itros|[Uu]n|[Oo]nzas)$',elem)

                            if len(t)==1:
                                saco2.append(lp[a])              # Si coincide, agregar a saco2
                            elif len(t1)==1:        
                                saco2.append(lp[a-1]+' '+lp[a])  # de lo contrario, juntarlo con el elemento anterior y agregarlo
                            a=a+1                                # a saco2
                    else:
                        saco2.append(q[0])
                        
                    if len(v)==0 or len(v1)==0:
                        saco2.append(q[0])
                    
                elif len(u)>0:
                    saco2.append('Pack')
            else:
                saco2.append(q[0])
            
            if saco2[0]=='Pack' or saco2[0]=='Mix':
                saco3.append('Metro')
                saco3.append(saco1[0])
            else:
                saco1=saco1[0].replace(saco2[0],'')
                saco3.append('Metro')
                saco3.append(saco1)
                
        elif e==i[1]:
            e=e.split()
            for q in e:
                z=re.findall('\d.+',q)
                if len(z)>0:
                    saco3.append(float(z[0]))
            
    saco3.append(saco2[0])
    total1.append(saco3)        

## Longitud de los datos
# len(total1

# Se crea dataframe para estructurar datos con las columnas Nombre-Precio(S/.)-Cantidad

columnas=['Local','Nombre','Precio','Cantidad']
Metro=pd.DataFrame(data=total1,columns=columnas)           

driver.close()
Metro

,Local,Nombre,Precio,Cantidad
0,Metro,Avena Natural Santa Catalina,11.99,900g
1,Metro,Avena Con Manzana Instantánea Santa Catalina B...,4.79,150 g
2,Metro,Avena Tradicional Cuisine&Co Bolsa,4.15,300 g
3,Metro,Avena con Algarrobina Santa Catalina,5.39,280g
4,Metro,Avena Cereal Andino,5.90,270 g
...,...,...,...,...
113,Metro,Jabón Líquido para Manos 2 en 1 Avena & Miel B...,9.20,Pack
114,Metro,Cereal Mix Avena y Almendras Cuisine & Co Caja,11.90,380 gr
115,Metro,"Barras de Almendras, Hojuelas Avena y Frutos S...",31.90,Pack
116,Metro,"Barras de Almendras, Hojuelas Avena y Frutos S...",31.90,Pack


# Plaza Vea

In [23]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import xml.etree.ElementTree as ET
import time
import pandas as pd

# Función que selecciona la pantalla de todos los productos en general e individualmente
def it():
    timeout=25 #tiempo de espera por pagina
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'showcase-grid'))
    WebDriverWait(browser, timeout).until(element_present) #Para esperar que cargue toda la pantalla con todos los productos
    elem_list = browser.find_element(
        By.CLASS_NAME, "showcase-grid") #Buscando todos los productos
    items = elem_list.find_elements(
        By.XPATH, '//div[@class="HA Showcase Showcase--food"]') #Seleccionando cada producto
    return items

# Función que almacena los valores que nos interesan de cada producto
def alm(): 
    table = [] # en donde se almacera toda la pagina
    for item in it():
        prov = [] # en donde se almacera los valores de un solo producto
        
        pagina="PlazaVea"
        
        prov.append(pagina)
        
        try:
            title = item.find_element(By.CLASS_NAME, 'Showcase__content').get_attribute("title")
            prov.append(title) #Seleccionando el titulo del producto
        except:
            title = "Sin info"
            prov.append(title)
            pass #En caso de que no seleccione, que pase.
        
        try:
            price = item.find_element(By.CLASS_NAME, "Showcase__salePrice").get_attribute("data-price")
            prov.append(price) #Seleccionando el precio del producto
        except:
            price = "Sin info"
            prov.append(price)
            pass #En caso de que no seleccione, que pase.
        
        try:
            cantidad = item.find_element(By.CLASS_NAME, "Showcase__units-reference").text
            prov.append(cantidad) #Seleccionando la cantidad del producto
        except:
            cantidad = "Sin info"
            prov.append(cantidad)
            pass #En caso de que no seleccione, que pase.
        
        
        table.append(prov) #almacena todos los productos de una sola pagina
    return table

print("hecho") #Imprime hecho si todo corrio correctamente

hecho


In [24]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_argument('--incognito')

browser = webdriver.Chrome(public,options = options)

#Buscar el producto

url = "https://www.plazavea.com.pe/search/?_query="+producto #Aqui colocamos los valores de la pagina y la categoria
browser.get(url)

WebDriverWait(browser, 50)\
    .until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'Header__dropdown__iconList__login__notification')))\
    .click()

time.sleep(2)

WebDriverWait(browser, 50)\
    .until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'CustomModal__close')))\
    .click()

elem_n = browser.find_element(
        By.CLASS_NAME, "pagination__nav") #Buscando todos los productos
itemx = elem_n.find_elements(
    By.CLASS_NAME, 'pagination__item') #Seleccionando cada producto

paginas = []

for item in itemx:
    paginas.append(item.text)

paginas2 = []
r=0
while r < len(paginas):
    if paginas[r].isdigit() == True:
        paginas2.append(paginas[r])
    r+=1

x=paginas2[len(paginas2)-1]
print("Cantidad de pagina", x)
n=0

datos=[]

while n<=int(x):
    try:
        n+=1
        WebDriverWait(browser, 15)\
            .until(EC.element_to_be_clickable((By.XPATH,
                                              '/html/body/div[4]/main/div/div[2]/div/section[2]/div[2]/div/div[3]/div/div[4]/div/div/span['+str(n)+']')))\
            .click()
        k=alm()
        print(len(k),x)
        datos.extend(k)
    except:
        break
#print(datos)

import pandas as pd

PlazaVea=pd.DataFrame(datos,columns = ['Local','Nombre','Precio','Cantidad'])
browser.close()
PlazaVea

<ipython-input-24-4a9894a0060a>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(public,options = options)


Cantidad de pagina 3
50 3
50 3
1 3


,Local,Nombre,Precio,Cantidad
0,PlazaVea,Avena Clásica 3 OSITOS Premium Bolsa 900g,11.90,Bolsa 900g
1,PlazaVea,Avena QUAKER Tradicional Bolsa 900g,14.50,Bolsa 900g
2,PlazaVea,Quinua Avena 3 OSITOS Bolsa 900g,10.90,Bolsa 900g
3,PlazaVea,Avena Instantánea Regular BELL'S Bolsa 1Kg,9.90,Bolsa 1kg
4,PlazaVea,Avena SANTA CATALINA Bolsa 1Kg,11.99,Bolsa 1kg
...,...,...,...,...
96,PlazaVea,Avena MILLER'S en Hojuelas sabor Canela Doypac...,5,Doypack 300g
97,PlazaVea,Avena Precocida BELL'S Bolsa 170g,1.40,Bolsa 170g
98,PlazaVea,Chía Avena GRANO DE ORO Hojuelas Precocidas Bo...,2.80,Bolsa 170g
99,PlazaVea,Cereal ANGEL Trigo integral de avena y maíz sa...,7.70,Bolsa 140g


In [34]:
PlazaVea.info()
#PlazaVea['Precio'] = pd.to_numeric(PlazaVea['Precio'],errors = 'coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Local     101 non-null    object 
 1   Nombre    101 non-null    object 
 2   Precio    101 non-null    float64
 3   Cantidad  101 non-null    object 
dtypes: float64(1), object(3)
memory usage: 3.3+ KB



# Tottus


In [27]:
# PRUEBA FINAL (todos los productos de Tottus)

# Se importan librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random
from time import sleep
from selenium.webdriver.chrome.service import Service

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(public,options = options)

# Se asigna la pagina web a scrapear
driver.get("https://www.tottus.com.pe/") 

# Selecciona y escribe el producto en el buscador Tottus 
try:
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#catalystSearchBar'))).click()
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input#inputSearchBar'))).send_keys(producto)
    #sleep(1)
    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.jsx-2851002401 searchIcon'.replace(' ','.')))).click()
except:
    WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,'input.jsx-3290144217 desktopSearchBar'.replace(' ','.')))).send_keys(producto)
    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,'button.jsx-3290144217 searchIcon'.replace(' ','.')))).click()


datos3=[]

while True:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,'//li[@class="product"]')))
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    sleep(1)

    elementos=driver.find_elements(By.XPATH,'//li[@class="product"]')
    for elem in elementos:
        saco=[]
        try:
            nombre= elem.find_element("xpath",'.//h2[@class="jsx-369422875 name title mini"]').text
            precio= elem.find_element("xpath",'.//span[@class="list price medium cmrPrice "]').text
            cantidad= elem.find_element("xpath",'.//h3[@class="jsx-369422875 extra text descriptive"]').text
            print(nombre)
            print(precio)
            print(cantidad)
            saco.append(nombre)
            saco.append(precio)
            saco.append(cantidad)
            datos3.append(saco)
        except:
            precio = 'NO DISPONIBLE'
            producto = 'NO DISPONIBLE'

    driver.execute_script("window.scrollBy(0, -800);")
    sleep(1)
    try:
        WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.XPATH,'//li[@class="next"]'))).click()
        sleep(2)
    except:
        break

<ipython-input-27-57207b2c2084>:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(public,options = options)


KIWICHA AVENA 3 OSITOS 270 GR
s/ 4.9 UN
3-OSITOS - BOLSA 270 GR
AVENA CON CEREALES ANDINOS 3 OSITOS 270 GR
s/ 4.9 UN
3-OSITOS - BOLSA 270 GR
AVENA GRANO DE ORO 370 GR
s/ 4.2 UN
GRANO-DE-ORO - BOLSA 370 GR
KIWICHA AVENA 3 OSITOS 170 GR
s/ 3.6 UN
3-OSITOS - BOLSA 170 GR
AVENA CON CHÍA GRANO DE ORO 170 GR
s/ 2.9 UN
GRANO-DE-ORO
QUINUA AVENA GRANO DE ORO 370 GR
s/ 4.9 UN
GRANO-DE-ORO - BOLSA 370 GR
AVENA CON CHÍA EN HOJUELAS GRANO DE ORO 370 GR
s/ 4.9 UN
GRANO-DE-ORO - BOLSA 370 GR
AVENA HOJUELAS PRECOCIDAS TOTTUS 1KG
s/ 9.9 UN
TOTTUS - BOLSA 1 KG
HOJUELAS PRECOCIDAS DE AVENA CON MACA GRANO DE ORO 370 GR
s/ 4.9 UN
GRANO-DE-ORO - BOLSA 370 GR
HOJUELAS DE QUINUA MOLICUSCO 200 GR
s/ 9.25 UN
MOLICUSCO - UNIDAD
HOJUELAS DE KIWICHA MOLICUSCO 200 GR
s/ 7.28 UN
MOLICUSCO - UNIDAD
AVENA INSTANTÁNEA TOTTUS 500 GR
s/ 8 UN
TOTTUS - BOLSA 500 GR
AVENA CHÍA EN HOJUELAS TOTTUS 380 GR
s/ 4 UN
TOTTUS - 380 GR
AVENA PREMIUM 3 OSITOS PRECOCIDA 900 GR
s/ 11.9 UN
3-OSITOS - 900 GR
AVENA MIX SUPERFOOD HUELLA VE

TimeoutException: Message: 


In [28]:

# LIMPIEZA Y ESTRUCTURACIÓN DE DATOS DESCARGADOS (Tottus)

# Se importan librerias
import re
import pandas as pd

total3=[]

for i in datos3:
    saco1=[]   # nombre-precio-cantidad
    saco1.append('Tottus')
    for e in i:
        if e==i[0]:
            saco1.append(e)
        if e==i[1]:
            e=e.split()
            for q in e:
                z=re.findall('\d.+',q)
                z1=re.findall('\d+',q)
                if len(z)>0: 
                    saco1.append(float(z[0]))
                elif len(z1)>0:
                    saco1.append(float(z1[0]))
        if e==i[2]:
            saco1.append(e)

    total3.append(saco1)        

# Se crea dataframe con datos ya estructurados con las columnas Nombre-Precio(S/.)-Cantidad

columnas=['Local','Nombre','Precio','Cantidad']
Tottus=pd.DataFrame(data=total3,columns=columnas)           
driver.close()
Tottus

,Local,Nombre,Precio,Cantidad
0,Tottus,KIWICHA AVENA 3 OSITOS 270 GR,4.90,3-OSITOS - BOLSA 270 GR
1,Tottus,AVENA CON CEREALES ANDINOS 3 OSITOS 270 GR,4.90,3-OSITOS - BOLSA 270 GR
2,Tottus,AVENA GRANO DE ORO 370 GR,4.20,GRANO-DE-ORO - BOLSA 370 GR
3,Tottus,KIWICHA AVENA 3 OSITOS 170 GR,3.60,3-OSITOS - BOLSA 170 GR
4,Tottus,AVENA CON CHÍA GRANO DE ORO 170 GR,2.90,GRANO-DE-ORO
5,Tottus,QUINUA AVENA GRANO DE ORO 370 GR,4.90,GRANO-DE-ORO - BOLSA 370 GR
6,Tottus,AVENA CON CHÍA EN HOJUELAS GRANO DE ORO 370 GR,4.90,GRANO-DE-ORO - BOLSA 370 GR
7,Tottus,AVENA HOJUELAS PRECOCIDAS TOTTUS 1KG,9.90,TOTTUS - BOLSA 1 KG
8,Tottus,HOJUELAS PRECOCIDAS DE AVENA CON MACA GRANO DE...,4.90,GRANO-DE-ORO - BOLSA 370 GR
9,Tottus,HOJUELAS DE QUINUA MOLICUSCO 200 GR,9.25,MOLICUSCO - UNIDAD


#  Comparación

In [35]:
# Ordena de menor a mayor de
M_orden=Metro.sort_values(by='Precio')
P_orden=PlazaVea.sort_values(by='Precio')
T_orden=Tottus.sort_values(by='Precio')
# Se selecciona el primer y ultimo producto de la tabla anterior
M_altobajo= M_orden.iloc[[0,-1]] 
P_altobajo= P_orden.iloc[[0,-1]]
T_altobajo= T_orden.iloc[[0,-1]]
# Se crea nueva columna que indica la magnitud del precio (más alto y más bajo)
T_altobajo=T_altobajo.assign(Precio_mag =['Precio más Bajo','Precio más Alto'])
P_altobajo=P_altobajo.assign(Precio_mag =['Precio más Bajo','Precio más Alto'])
M_altobajo=M_altobajo.assign(Precio_mag =['Precio más Bajo','Precio más Alto'])
T_altobajo.index = ['0', '1']
P_altobajo.index = ['0', '1']
M_altobajo.index = ['0', '1']

# Se unen las tablas (pd.concat) y se muestra de forma horizontal
comparacion1=pd.concat([M_altobajo,P_altobajo,T_altobajo],axis=1)      # Comparación en columnas
#comparacion=pd.concat([M_altobajo,T_altobajo],axis=0)     # Comparación en filas

comparacion1

,Local,Nombre,Precio,Cantidad,Precio_mag,Local,Nombre,Precio,Cantidad,Precio_mag,Local,Nombre,Precio,Cantidad,Precio_mag
0,Metro,Quinua Avena 3 Ositos Bolsa,1.99,170 g,Precio más Bajo,PlazaVea,Hojuelas de Avena con Plátano BELL'S Bolsa 35g,1.0,Bolsa 35g,Precio más Bajo,Tottus,AVENA CON CHÍA GRANO DE ORO 170 GR,2.9,GRANO-DE-ORO,Precio más Bajo
1,Metro,Jabón de Glicerina Simond's Avena,36.30,1L,Precio más Alto,PlazaVea,"Fibramax con Psyllium, Fibra de Trigo y Avena ...",77.0,Sin info,Precio más Alto,Tottus,HOJUELAS DE AVENA QUAKER QUICK 1.19 KG,27.0,QUAKER - UNIDAD,Precio más Alto


In [36]:
# Comparación desde el precio más bajo al precio más alto en todos los productos encontrados en las dos localidades
comparacion2=pd.concat([M_orden,P_orden,T_orden],axis=0) 
comparacion2.sort_values(by='Precio')

,Local,Nombre,Precio,Cantidad
46,PlazaVea,Hojuelas de Avena con Plátano BELL'S Bolsa 35g,1.00,Bolsa 35g
28,PlazaVea,Avena TRES OSITOS Clásica Bolsa 120g,1.39,Bolsa 120g
97,PlazaVea,Avena Precocida BELL'S Bolsa 170g,1.40,Bolsa 170g
75,PlazaVea,Harina de Avena BELL'S Piña Bolsa 80g,1.40,Bolsa 80g
52,PlazaVea,Harina de Avena BELL'S Vainilla Bolsa 80g,1.40,Bolsa 80g
...,...,...,...,...
105,Metro,Barras de Almendras Hojuelas Avena Sabor Choco...,31.90,Pack
115,Metro,"Barras de Almendras, Hojuelas Avena y Frutos S...",31.90,Pack
98,Metro,Loción Corporal Nivea Naturally Good Avena Nut...,32.90,350ml
39,Metro,Jabón de Glicerina Simond's Avena,36.30,1L


In [57]:
# Envío de de los resultados al correo ingresado anteriormente

from pretty_html_table import build_table
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import getpass

# Se establece un usuario y contraseña por defecto quien va a enviar el correo
# usuario: jossemen01@gmail.com
# contraseña: mxszphkwuhexmfue       <- contraseña generado para aplicaciones autorizadas en la cuenta gmail quien envía.

sender_address = 'usuario123321v1@gmail.com'
sender_pass = 'Usuario01'
receiver_address = correo

# El cuerpo del mensaje puede ser html, json, texto puro
html1 = build_table(comparacion1, 'blue_light')
html2 = build_table(comparacion2, 'blue_light')

body= 'Aquí se presentan los resultados con el precio más alto y más bajo' + html1 +'Aquí se presentan los resultados de los precios entre las tres localidades' + html2

message = MIMEMultipart("alternative")  # Tipos de mensaje multiparte como html
message['From'] = sender_address
message['To'] = receiver_address
message['Subject'] = 'Resultados de la comparación de producto en 3 localidades'   

message.attach(MIMEText(body, 'html'))   # plain -> texto puro(plano)   html-> formato html  json...
# Establece el tipo de correo y su puerto (587)
session = smtplib.SMTP('smtp.gmail.com', 587)
# Inicia sesión
session.starttls()
session.login(sender_address, sender_pass)
text = message.as_string()
# Envío de correo
session.sendmail(sender_address, receiver_address, text)
session.quit()

print('Los resultados de la comparación fueron enviados con éxito a su correo')

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials bj18-20020a0561220e5200b00378d82b16f3sm4259771vkb.54 - gsmtp')

In [ ]:
usuario123321v1
Usuario01